## Language Model Representations of Ambiguous (Spanish) Nouns in Context

Here, we load a monolingual Spanish-trained large language model (LLM) called [BETO](https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased), and we use it to compute vector representations for target ambiguous Spanish nouns. To do so, we also load a dataframe of sentence pairs in Spanish, where each pair contains a target ambiguous noun whose sense is disambiguated by either an adjective or a verb (termed context cue). This context cue marks the only difference across a given pair of sentences. Context cues have been chosen such that sometimes the sentence pair evokes the same sense for the target word, **or** evokes different (homonymous or polysemous) senses for the target word. 

We run each (tokenized version of each) sentence through BETO, and extract the vector representation, or embedding, for the target noun from each of BETO's layers. We then compute and store the cosine distances between the target word embeddings from the first and second sentences of the pair. 

In [1]:
%reset
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # makes figs nicer!

import functools
import itertools
import os
import torch
import transformers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from scipy.spatial.distance import cosine
from tqdm.notebook import tqdm
from transformers import AutoTokenizer


sns.set(style='whitegrid',font_scale=1.2)

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


### define useful custom functions

In [2]:
### Define useful custom functions to ...

### ... find the target tokens within tokenized sequence
def find_sublist_index(mylist, sublist):
    """Find the first occurence of sublist in list.
    Return the start and end indices of sublist in list"""

    for i in range(len(mylist)):
        if mylist[i] == sublist[0] and mylist[i:i+len(sublist)] == sublist:
            return i, i+len(sublist)
    return None

@functools.lru_cache(maxsize=None)  # This will cache results, handy later...


### ... grab the embeddings for your target tokens
def get_embedding(model, tokenizer, sentence, target, layer, device):
    """Get a token embedding for target in sentence"""
    
    # Tokenize sentence
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    
    # Tokenize target
    target_enc = tokenizer.encode(target, return_tensors="pt",
                                  add_special_tokens=False).to(device)
    
    # Get indices of target in input tokens
    target_inds = find_sublist_index(
        inputs["input_ids"][0].tolist(),
        target_enc[0].tolist()
    )

    # Run model
    with torch.no_grad():
        output = model(**inputs)
        hidden_states = output.hidden_states

    # Get layer
    selected_layer = hidden_states[layer][0]

    #grab just the embeddings for your target word's token(s)
    token_embeddings = selected_layer[target_inds[0]:target_inds[1]]

    #if a word is represented by >1 tokens, take mean
    #across the multiple tokens' embeddings
    embedding = torch.mean(token_embeddings, dim=0)
    
    return embedding

### load your model and tokenizer

In [3]:
### Models
MODELS = ["dccuchile/bert-base-spanish-wwm-cased",
         "bert-base-multilingual-cased",
         "xlm-roberta-base",
         "dccuchile/albert-tiny-spanish",
          "dccuchile/albert-base-spanish",
          "dccuchile/albert-large-spanish",
          "dccuchile/albert-xlarge-spanish",
          "dccuchile/albert-xxlarge-spanish",
          "PlanTL-GOB-ES/roberta-base-bne",
          "PlanTL-GOB-ES/roberta-large-bne",
          "dccuchile/bert-base-spanish-wwm-uncased", ### TODO: Add
         "dccuchile/distilbert-base-spanish-uncased"]

### TODO: Other models?

In [3]:
### Define the url path to BETO

mpath = "dccuchile/bert-base-spanish-wwm-cased" 
mpath = "dccuchile/bert-base-spanish-wwm-uncased"
## Use "bert-base-multilingual-cased" for multlingual BERT comparison


### Decide which device you want the models to run in
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

### Load your model & tokenizer

# model = transformers.AutoModelForCausalLM.from_pretrained(mpath,output_hidden_states=True)
model = transformers.AutoModel.from_pretrained(mpath,output_hidden_states=True)
model.to(device) # allocate model to desired device

tokenizer = transformers.AutoTokenizer.from_pretrained(mpath)  



config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/Users/seantrott/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

### load the dataframe of sentence pairs

In [4]:
df = pd.read_csv("../data/raw/items/sawc_sentence_pairs.csv")

In [5]:
df.shape[0] # number of sentence pairs

812

### compute cosine distances

for each target word within a pair of sentences, for each model layer

In [6]:
### Getting number of hidden layers 
### TODO: Double-check for all configurations
n_layers = model.config.num_hidden_layers
n_layers

12

In [7]:
### Get the number of layers directly from the model specifications
"""
if mpath == "xlm-roberta-base":
    n_layers = len(model.base_model.encoder.layer)
elif mpath =="dccuchile/distilbert-base-spanish-uncased":
    n_layers = len(model.base_model.transformer.layer)
elif mpath == "dccuchile/albert-tiny-spanish":
    n_layers = model.config.num_hidden_layers
else:
    n_layers = len(model.bert.encoder.layer)
"""

'\nif mpath == "xlm-roberta-base":\n    n_layers = len(model.base_model.encoder.layer)\nelif mpath =="dccuchile/distilbert-base-spanish-uncased":\n    n_layers = len(model.base_model.transformer.layer)\nelif mpath == "dccuchile/albert-tiny-spanish":\n    n_layers = model.config.num_hidden_layers\nelse:\n    n_layers = len(model.bert.encoder.layer)\n'

In [8]:
results = []

for layer in range(n_layers+1): # `range` is non-inclusive for the last value of interval
    for (ix, row) in tqdm(df.iterrows(), total=df.shape[0]):

        ### Get embeddings for S1 and S2
        
        ## TODO: For RoBERTa
        if mpath in ["PlanTL-GOB-ES/roberta-base-bne", "PlanTL-GOB-ES/roberta-large-bne"]:
            target = " {w}".format(w = row['Word'])
        else:
            target = row['Word']
        
        s1 = get_embedding(model, tokenizer, row['Sentence_1'], target,layer, device)
        s2 = get_embedding(model, tokenizer, row['Sentence_2'], target,layer, device)

        ### Now calculate cosine distance 
        #.  note, tensors need to be copied to cpu to make this run
        #.  still faster to do this copy than to just have everything
        #.  running on the cpu
        if device.type == "mps":  
            model_cosine = cosine(s1.cpu(), s2.cpu())
            
        else: 
            model_cosine = cosine(s1, s2)


        ### Figure out how many tokens you're
        ### comparing across sentences
        n_tokens_s1 = len(tokenizer.encode(row['Sentence_1']))
        n_tokens_s2 = len(tokenizer.encode(row['Sentence_1']))

        ### Add to results dictionary
        results.append({
            'Sentence_1': row['Sentence_1'],
            'Sentence_2': row['Sentence_2'],
            'Word': row['Word'],
            'Same_sense': row['Same_sense'],
            'Distance': model_cosine,
            'Layer': layer,
            'S1_ntokens': n_tokens_s1,
            'S2_ntokens': n_tokens_s2
        })
        
df_results = pd.DataFrame(results)
df_results['token_diffs'] = np.abs(df_results['S1_ntokens'].values-df_results['S2_ntokens'].values)


  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

...

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

### hurray! save your cosine distances to load in the next notebook

In [9]:
### Save your cosine distance results 

savepath = "../data/processed/models/"
if not os.path.exists(savepath): 
    os.mkdir(savepath)
    

### Replace "beto" with name of model
df_results.to_csv(os.path.join(savepath,"beto-uncased_sawc_distances.csv"), index=False)
# df_results.to_csv(os.path.join(savepath,"beto_sawc_distances.csv"), index=False)